In [7]:
import pandas as pd
import numpy as np
from collections import Counter, defaultdict, namedtuple

# Powerlifting Data
I grabbed a powerlifting dataset from kaggle (https://www.kaggle.com/open-powerlifting/powerlifting-database).  

It contains two files - (1) meet information and (2) meet results

For those of you that are not familiar with powerlifting - athlete's are judged on 3 lifts: squat, bench, and deadlift.  Each athlete has 3 attempts at each lift, and are competing to achieve the best overall score in a meet.  The overall score is calculated by adding together their best squat, bench and deadlift.  

Our goal - answer the following questions:
1. What federation has held the most meets to date (past 3 years)?
2. What state has held the most meets to date (past 3 years)
3. Can we find the top squat, bench, deadlift and total for each weight category?  


Note: Since we are trying to become more familiar with python Collections, we will try to utilize Counters, defaultdicts, and namedtuples to answer these questions.  

## Meets

In [128]:
# load the data
meets = pd.read_csv('./meets.csv')
meets['Date'] = pd.to_datetime(meets.Date)
meets['Year'] = meets.Date.dt.year
meets.head()

,MeetID,MeetPath,Federation,Date,MeetCountry,MeetState,MeetTown,MeetName,Year
0,0,365strong/1601,365Strong,2016-10-29,USA,NC,Charlotte,2016 Junior & Senior National Powerlifting Cha...,2016
1,1,365strong/1602,365Strong,2016-11-19,USA,MO,Ozark,Thanksgiving Powerlifting Classic,2016
2,2,365strong/1603,365Strong,2016-07-09,USA,NC,Charlotte,Charlotte Europa Games,2016
3,3,365strong/1604,365Strong,2016-06-11,USA,SC,Rock Hill,Carolina Cup Push Pull Challenge,2016
4,4,365strong/1605,365Strong,2016-04-10,USA,SC,Rock Hill,Eastern USA Challenge,2016


In [129]:
for year in sorted(meets.Year.unique(), reverse=True)[:3]:
    print(year)
    print('-----')
    
    # what is the most frequent federation?
    most_frequent_federation = Counter(meets[meets.Year==year].Federation.values)
    for fed in most_frequent_federation.most_common(5):
        print(fed)
    

2018
-----
('USAPL', 21)
('USPA', 10)
('CPU', 3)
('GPC-AUS', 3)
('RPS', 3)
2017
-----
('THSPA', 341)
('USAPL', 299)
('USPA', 252)
('NSF', 100)
('PA', 88)
2016
-----
('THSPA', 329)
('USAPL', 249)
('USPA', 213)
('NSF', 115)
('RPS', 77)


In [130]:
for year in sorted(meets.Year.unique(), reverse=True)[:3]:
    print(year)
    print('-----')
    
    # what is the most frequent federation?
    most_frequent_state = Counter(meets[meets.Year==year].MeetState.values)
    for state in most_frequent_state.most_common(5):
        print(state)
    

2018
-----
('TX', 4)
('CA', 4)
(nan, 3)
('MA', 3)
('WI', 3)
2017
-----
('TX', 392)
(nan, 197)
('CA', 86)
('ON', 49)
('NY', 47)
2016
-----
('TX', 382)
(nan, 223)
('CA', 68)
('PA', 42)
('OH', 42)


## Meet Results

In [103]:
lifting_results = pd.read_csv('./openpowerlifting.csv')
lifting_results.head()

,MeetID,Name,Sex,Equipment,Age,Division,BodyweightKg,WeightClassKg,Squat4Kg,BestSquatKg,Bench4Kg,BestBenchKg,Deadlift4Kg,BestDeadliftKg,TotalKg,Place,Wilks
0,0,Angie Belk Terry,F,Wraps,47.0,Mst 45-49,59.60,60,NaN,47.63,NaN,20.41,NaN,70.31,138.35,1,155.05
1,0,Dawn Bogart,F,Single-ply,42.0,Mst 40-44,58.51,60,NaN,142.88,NaN,95.25,NaN,163.29,401.42,1,456.38
2,0,Dawn Bogart,F,Single-ply,42.0,Open Senior,58.51,60,NaN,142.88,NaN,95.25,NaN,163.29,401.42,1,456.38
3,0,Dawn Bogart,F,Raw,42.0,Open Senior,58.51,60,NaN,NaN,NaN,95.25,NaN,NaN,95.25,1,108.29
4,0,Destiny Dula,F,Raw,18.0,Teen 18-19,63.68,67.5,NaN,NaN,NaN,31.75,NaN,90.72,122.47,1,130.47


Powerlifters can lift 'equipped' or 'raw'.  Raw lifters will not use equipment to assist them in lifting (such as knee wraps or specialized singlets).  We will only look at raw results for this analysis.  

In [104]:
lifting_results.Equipment.unique()

array(['Wraps', 'Single-ply', 'Raw', 'Multi-ply', 'Straps'], dtype=object)

In [105]:
# drop equipped lifters from this analysis
lifting_results = lifting_results[lifting_results.Equipment == 'Raw']
lifting_results.head()

,MeetID,Name,Sex,Equipment,Age,Division,BodyweightKg,WeightClassKg,Squat4Kg,BestSquatKg,Bench4Kg,BestBenchKg,Deadlift4Kg,BestDeadliftKg,TotalKg,Place,Wilks
3,0,Dawn Bogart,F,Raw,42.0,Open Senior,58.51,60,NaN,NaN,NaN,95.25,NaN,NaN,95.25,1,108.29
4,0,Destiny Dula,F,Raw,18.0,Teen 18-19,63.68,67.5,NaN,NaN,NaN,31.75,NaN,90.72,122.47,1,130.47
6,0,Maureen Clary,F,Raw,60.0,Mst 60-64,67.31,67.5,NaN,124.74,NaN,95.25,NaN,163.29,383.28,1,391.98
7,0,Maureen Clary,F,Raw,60.0,Open Senior,67.31,67.5,NaN,124.74,NaN,95.25,NaN,163.29,383.28,1,391.98
9,0,Priscilla Sweat Pardue,F,Raw,52.0,Senior,65.95,67.5,NaN,NaN,NaN,NaN,NaN,108.86,108.86,1,112.99


We also want to look at results that are complete.  We particularly interested in Name, Sex, WeightClassKg, BestSquatKg, BestBenchKg, BestDeadliftKg and Totalkg.

In [133]:
# merge meet information into dataframe
lifting_results = pd.merge(lifting_results, meets, how='left', left_on='MeetID', right_on='MeetID')

In [134]:
# take a look at missing data
lifting_results.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 186317 entries, 0 to 186316
Data columns (total 25 columns):
MeetID            186317 non-null int64
Name              186317 non-null object
Sex               186317 non-null object
Equipment         186317 non-null object
Age               99001 non-null float64
Division          177538 non-null object
BodyweightKg      185995 non-null float64
WeightClassKg     185317 non-null object
Squat4Kg          454 non-null float64
BestSquatKg       136210 non-null float64
Bench4Kg          1123 non-null float64
BestBenchKg       172729 non-null float64
Deadlift4Kg       1518 non-null float64
BestDeadliftKg    155816 non-null float64
TotalKg           179914 non-null float64
Place             185574 non-null object
Wilks             179642 non-null float64
MeetPath          186317 non-null object
Federation        186317 non-null object
Date              186317 non-null datetime64[ns]
MeetCountry       186317 non-null object
MeetState         1

In [135]:
# drop missing data within our specific fields
lifting_results.dropna(subset=['WeightClassKg', 'BestSquatKg', 'BestBenchKg', 'BestDeadliftKg', 'TotalKg', 'Year'], inplace=True)

Powerlifting also have various weight classes - which are different for male and female.  Let's ensure we only utilize the current IPF weight classes.

In [136]:
# list weight classes in the current dataset
lifting_results[lifting_results.Sex == 'F'].WeightClassKg.unique()

array(['67.5', '75', '82.5', '90', '52', '60', '56', '90+', '110', '35',
       '40', '44', '48', '100', '57', '63', '72', '84', '84+', '47', '43',
       '59', '100+', '53', '80', '74', '105', '66', '83', '110+', '75+',
       '30', '93', '34'], dtype=object)

In [137]:
# Define IPF weight classes
mens_weight_classes = ['53', '59', '66', '74', '83', '93', '105', '120', '120+']
womens_weight_classes = ['43', '47', '52', '57', '63', '72', '84', '84+']

With the data prepped, let's load our data in to a defaultdict for organization.  We will group the meet results by Sex & Weight class.

In [140]:
# create a namedtuple to represent each lift
Lift = namedtuple('Lift', ['name', 'squat', 'bench', 'deadlift', 'total', 'year'])

# create a namedtuple ro represent each weight class / sex combindation
DClass = namedtuple('DClass', ['weight_class', 'sex'])

# create defaultdict to manage results
results = defaultdict(list)

for index, row in lifting_results.iterrows():
        
    # we will only record a result if the lifter is part of our recognized weight classes
    if ((row.Sex == 'F') & (row.WeightClassKg in womens_weight_classes)) | \
    ((row.Sex =='M') & (row.WeightClassKg in mens_weight_classes)):
            key = DClass(row.WeightClassKg, row.Sex)
            value = Lift(row.Name, row.BestSquatKg, row.BestBenchKg, row.BestDeadliftKg, row.TotalKg, row.Year)               
            
            results[key].append(value)

In [141]:
# verify our results
results[DClass('53', 'M')]

[Lift(name='Étienne Bergeron', squat=77.5, bench=40.0, deadlift=85.0, total=202.5, year=2011),
 Lift(name='Étienne Bergeron', squat=85.0, bench=52.5, deadlift=125.0, total=262.5, year=2012),
 Lift(name='André Albert', squat=102.5, bench=67.5, deadlift=120.0, total=290.0, year=2013),
 Lift(name='Tom-Alexandre Leroux-Moreau', squat=55.0, bench=32.5, deadlift=72.5, total=160.0, year=2013),
 Lift(name='Bryce Booker', squat=75.0, bench=40.0, deadlift=95.0, total=210.0, year=2014),
 Lift(name='Owen Jones', squat=52.5, bench=40.0, deadlift=85.0, total=177.5, year=2015),
 Lift(name='John Pineda', squat=120.0, bench=92.5, deadlift=142.5, total=355.0, year=2015),
 Lift(name='Ryan MacKay', squat=45.0, bench=27.5, deadlift=75.0, total=147.5, year=2016),
 Lift(name='John Pineda', squat=122.5, bench=100.0, deadlift=162.5, total=385.0, year=2016),
 Lift(name='Ryan MacKay', squat=52.5, bench=32.5, deadlift=80.0, total=165.0, year=2016),
 Lift(name='Ryan MacKay', squat=55.0, bench=35.0, deadlift=87.5, 

With the dictionary created, we just need to find the top result per weight class and print the result.  I've created a helper function to print the results below:

In [153]:
def print_best_lifts(values):

    # sort the values
    best = sorted(values, key=lambda values: values.squat, reverse=True)[0]    
    print('Squat     {}kg by {} ({})'.format(best.squat, best.name, best.year))

    # sort the values
    best = sorted(values, key=lambda values: values.bench, reverse=True)[0]
    print('Bench     {}kg by {} ({})'.format(best.bench, best.name, best.year))

    # sort the values
    best = sorted(values, key=lambda values: values.deadlift, reverse=True)[0]
    print('Deadlift  {}kg by {} ({})'.format(best.deadlift, best.name, best.year))

    # sort the values
    best = sorted(values, key=lambda values: values.total, reverse=True)[0]    
    print('Total     {}kg by {} ({})'.format(best.total, best.name, best.year))

In [154]:
# loop through womens weight classes
for w in womens_weight_classes:
    print('Sex: F | Weight Class: {}kg'.format(w))
    print('-------------------------------')

    # create key
    key = DClass(w, 'F')
    
    # get values from dictionary
    values = results[key]
    
    # print results
    print_best_lifts(values)    
    print()
    
# loop through mens weight classes
for m in mens_weight_classes:
    print('Sex: M | Weight Class: {}kg'.format(m))
    print('-------------------------------')

    # create key
    key = DClass(m, 'M')
    
    # get values from dictionary
    values = results[key]
    
    # print results
    print_best_lifts(values)    
    print()

Sex: F | Weight Class: 43kg
-------------------------------
Squat     102.5kg by Marina Shostachuk (2016)
Bench     70.0kg by Marina Shostachuk (2016)
Deadlift  135.0kg by Kozue Nemoto (2015)
Total     282.5kg by Marina Shostachuk (2016)

Sex: F | Weight Class: 47kg
-------------------------------
Squat     152.5kg by Wei-Ling Chen (2015)
Bench     95.5kg by Hanna Rantala (2017)
Deadlift  175.5kg by Heather Connor (2017)
Total     407.5kg by Wei-Ling Chen (2015)

Sex: F | Weight Class: 52kg
-------------------------------
Squat     162.5kg by Jenn Rotsinger (2017)
Bench     110.5kg by Febrose Tsiode (2017)
Deadlift  190.0kg by Jenn Rotsinger (2017)
Total     445.0kg by Jenn Rotsinger (2017)

Sex: F | Weight Class: 57kg
-------------------------------
Squat     174.5kg by Jennifer Millican (2017)
Bench     107.5kg by Hui-Chun Wu (2017)
Deadlift  193.0kg by Jennifer Millican (2017)
Total     463.0kg by Jennifer Millican (2017)

Sex: F | Weight Class: 63kg
-------------------------------
